# Differentiable Lotto Implementation
(based on the "Open-ended Learning in Symmetric Zero-sum Games" paper)

## 1. Introduction


**Differentiable Lotto** is inspired by continuous Lotto (Hart, S. Discrete Colonel Blotto and General Lotto games. Int J Game Theory, 36:441–460, 2008.). The game is defined over a fixed set $C$ of $c$ 'customers', each being a point in $\mathbb{R}^2$. An agent’s strategy $(p, v) = \{(p_1,v_1), ... , (p_k,v_k)\}$, distributes one unit of mass over $k$ servers, where each server is a point $v_i \in \mathbb{R}^2$. Roughly, given the strategies of two players, $(p,v)$, $(q,w)$, customers are softly assigned to the nearest servers, determining the agents’ payoffs.

More formally, the payoff is

$$
\phi((p,v), (q,w)) := \sum_{i,j=1}^{c,k} (p_j v_{ij} - q_j w_{ij}),$$

where the scalars $v_{ij}$ and $w_{ij}$ depend on the distance between customer $i$ and the servers:

$$(v_{i1}, ... , w_{ik}) := softmax(-||c_i - v_1||^2, ... , -||c_i - w_k||^2),$$

where $c_i$ is the coordinate of customer $i$.

The width of a cloud of points is the expected distance from the barycenter. We impose agents to have width equal one. We use gradient ascent as our oracle.

For this implmentation,

- $c = 9$ customers chosen uniformly at random in the square $[-1,1]^2$
- $k = 2$ servers
- $n = 500$ games

## 2. Code Implementation of the game

### 2.0 Importing all necessary packages

In [1]:
import numpy as np
from random import randint
from scipy.optimize import linprog

----------------

### 2.1 Initialization of initial strategies and fixed c customers

*random_point(lb = -1, ub = 1, n = 2)*:
- Input: Lower bound (lb), upper bound (ub) of each coordinate in $\mathbb{R}^n$ space
- Output: random point within the space

In [2]:
def random_point(lb = -1, ub = 1, n = 2):
    point = np.zeros((n,))
    for i in range(n):
        point[i] = np.random.uniform(lb, ub)
    return point

In [3]:
random_point()

array([0.36229518, 0.36394426])

*random_points_list(lb = -1, ub = 1, n = 2, k = 2)*:
- Input: $k$ number of points with lower bound (lb), upper bound (ub) of each coordinate in $\mathbb{R}^n$ space
- Output: list of $k$ random points within the space

In [4]:
def random_points_list(lb = -1, ub = 1, n = 2, k = 9):
    points_list = []
    for i in range(k):
        points_list.append(random_point(lb,ub,n))
    points_list = np.asarray(points_list)
    return points_list

In [5]:
random_points_list(k=9)

array([[ 0.07289642,  0.89895292],
       [-0.47312269,  0.08480472],
       [ 0.09138947, -0.36477769],
       [-0.09047958, -0.11435868],
       [ 0.95584869,  0.56662997],
       [-0.47810368, -0.0714073 ],
       [-0.79524905,  0.42234337],
       [ 0.53449333, -0.62198365],
       [-0.99056606,  0.8104629 ]])

*prob_dist(k = 2)*:
- Input: $k$
- Output: $k$ numbers that sum up to 1

In [6]:
def prob_dist(k = 2):
    pd = np.random.uniform(0,1,k)
    total = np.sum(pd)
    pd = pd/total
    return pd

In [7]:
prob_dist(k=2)

array([0.44830956, 0.55169044])

*initialise_strategy(lb = -1, ub = 1, n = 2, k = 2)*:
- Input: $k$ number of points with lower bound (lb), upper bound (ub) of each coordinate in $\mathbb{R}^n$ space
- Output: p_vector (distribution of k points) and v_vector (k points in $\mathbb{R}^n$)

In [8]:
def initialise_strategy(lb = -1, ub = 1, n = 2, k = 2):
    v_vector = random_points_list(lb, ub, n, k)
    p_vector = prob_dist(k)
    return p_vector, v_vector

In [9]:
initialise_strategy(lb = -1, ub = 1, n = 2, k = 2)

(array([0.40571654, 0.59428346]), array([[ 0.42913459, -0.56027163],
        [ 0.79665451, -0.23165189]]))

Take for example: Generate 2 players' strategies: (p,v) and (q,w) with k = 2 servers each. c = 9 Customers in $[-1,1]^2$ space

In [10]:
lb = -1 #lower bound of the space
ub = 1 #upper bound of the space
n = 2 #dimension of the space
k = 2 #number of servers
c = 9 #number of customers

p,v = initialise_strategy(lb, ub, n, k)
C = random_points_list(lb, ub, n, k = c)
print("Player's strategies distribution: " + np.array2string(p) + " with servers at " + np.array2string(v) + ".")
print("The customers are located at " + np.array2string(C) + ".")

Player's strategies distribution: [0.7109327 0.2890673] with servers at [[ 0.52377423 -0.13829908]
 [-0.64764852  0.00605504]].
The customers are located at [[-0.40930006 -0.11372282]
 [ 0.45471504 -0.40404271]
 [-0.19604131  0.21068581]
 [ 0.51727535 -0.21181064]
 [-0.89968634  0.06983997]
 [ 0.69187852 -0.18296335]
 [-0.61434465 -0.82354691]
 [ 0.40273431 -0.44720276]
 [-0.97642773 -0.3457468 ]].


------------------

### 2.2 Necessary functions for PSRO algorithm

*Nash_eq(A)*:
- Input: matrix A of a population
- Output: Nash equilibrium p

In [11]:
def Nash_eq(A):
    '''Input: matrix A of a population
       Output: Nash equilibrium
       Note that A need to be antisymmetric for this function to generate the right Nash p'''
    n = A.shape[0]
    A_ub = np.concatenate((-A.T,[np.ones(n,),-np.ones(n,)]), axis = 0)
    b_ub = np.append(np.zeros((n,)),[1,-1])
    soln = linprog(c = np.zeros((n,)), A_ub = A_ub, b_ub = b_ub, bounds = (0,1))
    return soln.x

RPS example:

In [12]:
A = np.array([[0,1,-1],[-1,0,1],[1,-1,0]])
print(Nash_eq(A))

[0.33333333 0.33333333 0.33333333]


*eval_matrix_1pop(popn)*:
- Input: list of strategies from 1 population of strategies where each strategy = $[p^i,v^i]$
- Output: evaluation matrix A (as described in Section 2 of the paper)

In [13]:
def eval_matrix_1pop(popn):
    '''Input: 1 population of strategies
       Output: evaluation matrix of that population'''
    n = len(popn)
    matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            matrix[i][j] = payoff(popn[i],popn[j])
    return matrix

*eval_matrix_2pop(popn)*:
- Input: lists of strategies from 2 population of strategies where each strategy = $[p^i,v^i]$ (no. of strategies from population A can differ from population B's)
- Output: evaluation matrix A (as described in Section 2 of the paper)

In [14]:
def eval_matrix_2pop(popn1,popn2):
    '''Input: 2 population of strategies
       Output: evaluation matrix of those 2 populations'''
    n = len(popn1)
    m = len(popn2)
    matrix = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            matrix[i][j] = payoff(popn1[i],popn2[j])
    return matrix

-----------------

### 2.3 PSRO$_N$ Implementation

In [15]:
def PSRO_N(popn, epoch = 100):
    strategies = popn
    n = len(strategies)
    for i in range(epoch):
        A = eval_matrix(strategies)
        nash_p = Nash_eq(A)
        new_strategy = oracle(nash_p, strategies)
        strategies.append(new_strategy)
        n = n + 1
        print("There are " + n + " strategies in the population now.")
    return strategies

--------------

### 2.4 Population Performance Measures

*rel_popn_perf(popn1, popn2)*:
- Input: lists of strategies from 2 population of strategies where each strategy = $[p^i,v^i]$ (no. of strategies from population A can differ from population B's)
- Output: relative population performance (defined in Definition 3 of Section 3.1 in th paper)

In [17]:
def rel_popn_perf(popn1, popn2):
    perf = 0
    A_1 = eval_matrix_1pop(popn1)
    A_2 = eval_matrix_1pop(popn2)
    A_12 = eval_matrix_2pop(popn1,popn2)
    p = Nash_eq(A_1).reshape(-1,1)
    q = Nash_eq(A_2).reshape(-1,1)
    perf = np.dot(np.dot(p.T,A_12),q)
    return perf

*eff_diversity(popn)*:
- Input: list of strategies from 1 population of strategies where each strategy = $[p^i,v^i]$
- Output: effective diversity of the population (defined in Definition 4 of Section 3.2 in the paper)

In [22]:
def eff_diversity(popn):
    d = 0
    A = eval_matrix_1pop(popn)
    p = Nash_eq(A).reshape(-1,1)
    A_plus = np.clip(A,0,None)
    d = np.dot(np.dot(p.T,A_plus),p)
    return d

--------------

In [ ]:
def softmax_fn(v,w,C,k,c):
    dist_v=np.zeros((c,k));
    dist_w=np.zeros((c,k));
    soft_v=np.zeros((c,k));
    soft_w=np.zeros((c,k));
    for i in range(c):
        for j in range(k):
            dist_v[i][j]=np.exp(-np.norm(C[i,:]-v[j,:]))
            dist_w[i][j]=np.exp(-np.norm(C[i,:]-w[j,:]))
        soft_v[i,:]=dist_v[i,:]/(np.sum(dist_v[i,:])+np.sum(dist_w[i,:]));
        soft_w[i,:]=dist_w[i,:]/(np.sum(dist_v[i,:])+np.sum(dist_w[i,:]));
    return soft_v,soft_w,dist_v,dist_w     

In [ ]:
def payoff(p,soft_v,q,soft_w):
    payoff=np.sum(np.dot(soft_v,p)-np.dot(soft_w,q))
    return payoff

In [ ]:
def oracle(popn,nash_p,eta,C,k,c):
    curr_agent_p=popn[-1][0];
    curr_agent_v=popn[-1][1];
    next_agent_p=curr_agent_p+eta*gradient_p(popn,nash_p,curr_agent_v,C,k,c);
    next_agent_v=curr_agent_v+eta*gradient_v(popn,nash_p,curr_agent_p,curr_agent_v,C,k,c);
    return next_agent_p,next_agent_v;

In [ ]:
def gradient_p(popn,nash_p,agent_v,C,k,c):
    sum_vec=np.zeros((k,1));
    for i in range(len(popn)):
        temp_agent_p=popn[i][0];
        temp_agent_p=popn[i][1];
        temp_v,temp_w,tempdist_v,tempdist_w=softmax_fn(agent_v,temp_agent_w,C,k,c);
        val_grad=np.sum(temp_v,axis=0) #sum the rows
        sum_vec=sum_vec+nash_p[i]*val_grad; #multiply w.r.t Nash p.
        
    return sum_vec;    

In [ ]:
def gradient_v(popn,nash_p,agent_p,agent_v,C,k,c): #agent_p, agent_v are strategies of the current agent "v_t of popn B_t"
    sum_mat=np.zeros((k,2));
    for i in range(len(popn)):
        temp_agent_p=popn[i][0];
        temp_agent_v=popn[i][1];
        temp_v,temp_w,tempdist_v,tempdist_w=softmax_fn(agent_v,temp_agent_v,C,k,c);
        val_grad_mat=calc_gradient_matrix(agent_p,agent_v,temp_agent_p,temp_agent_v,temp_v,temp_w,tempdist_v,tempdist_w,C,k,c) #sum the rows
        #To add the remaining parameters to calc_gradient_matrix function-See below.
        sum_mat=sum_mat+nash_p[i]*val_grad_mat; #multiply w.r.t Nash p.
        
    return sum_mat

In [ ]:
def calc_gradient_matrix(agent_p,agent_v,temp_agent_p,temp_agent_v,temp_v,temp_w,tempdist_v,tempdist_w,C,k,c):
    #agent_p - p vector of agent v_t
    #temp_agent_q- q vector of the "current opposition agent"
    #W- location matrix of "current opposition agent"
    #V- location matrix of agent v_t
    grad_mat=np.zeros((k,2));
    V=agent_v;
    W=temp_agent_v;
    for j in range(k):
      S_v=0;
      S_w=0;
      for i in range(c):
        for l in range(k):
            v_loc=V[j,:];
            S_v=S_v+agent_p[j]*grad_payoff(v_loc,j,l,1,i,temp_v,temp_w,dist_v,dist_w,C,k,c);
            S_w=S_w+temp_agent_p[j]*grad_payoff(v_loc,j,l,0,i,temp_v,temp_w,dist_v,dist_w,C,k,c);
        
         
      grad_mat[j,:]=S_v-S_w;  
      
    return grad_mat    

In [ ]:
def grad_payoff(v_loc,v_loc_idx,w_loc_idx,agent_bool,c_idx,temp_v,temp_w,dist_v,dist_w,C,k,c): #Gradient with respect to v_loc a location strategy of agent v.
    
    if(agent_bool==1): #Differentiating agent_v
        
     if(v_loc_idx~=w_loc_idx):
         grad=(2*(C[c_idx,:]-v_loc)*dist_v(c_idx,v_loc_idx)*(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:])-dist_v(c_idx,v_loc_idx)))/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))^2
     else:
        grad=(-2*(C[c_idx,:]-v_loc)*dist_v(c_idx,v_loc_idx)*dist_v(c_idx,w_loc_idx))/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))^2
    
    else:
        
     if(v_loc_idx~=w_loc_idx): #Differentiating agent_w
         grad=(2*(C[c_idx,:]-v_loc)*dist_v(c_idx,v_loc_idx)*(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:])-dist_w(c_idx,v_loc_idx)))/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))^2
     else:
         grad=(-2*(C[c_idx,:]-v_loc)*dist_v(c_idx,v_loc_idx)*dist_w(c_idx,w_loc_idx))/(np.sum(dist_v[c_idx,:])+np.sum(dist_w[c_idx,:]))^2   
    
    return grad    